In [1]:
# %load mnist_loader.py
"""
mnist_loader
~~~~~~~~~~~~
A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [2]:
# %load network.py

"""
network.py
~~~~~~~~~~
IT WORKS

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [3]:
training_data , validation_data , test_data = load_data_wrapper()

In [4]:
net = Network([784, 30, 10])

In [5]:
net.SGD(training_data , 30, 10, 3.0, test_data=test_data)

Epoch 0 : 7984 / 10000
Epoch 1 : 9185 / 10000
Epoch 2 : 9305 / 10000
Epoch 3 : 9356 / 10000
Epoch 4 : 9410 / 10000
Epoch 5 : 9419 / 10000
Epoch 6 : 9404 / 10000
Epoch 7 : 9427 / 10000
Epoch 8 : 9460 / 10000
Epoch 9 : 9449 / 10000
Epoch 10 : 9439 / 10000
Epoch 11 : 9489 / 10000
Epoch 12 : 9476 / 10000
Epoch 13 : 9442 / 10000
Epoch 14 : 9487 / 10000
Epoch 15 : 9483 / 10000
Epoch 16 : 9494 / 10000
Epoch 17 : 9475 / 10000
Epoch 18 : 9490 / 10000
Epoch 19 : 9507 / 10000
Epoch 20 : 9492 / 10000
Epoch 21 : 9489 / 10000
Epoch 22 : 9511 / 10000
Epoch 23 : 9495 / 10000
Epoch 24 : 9498 / 10000
Epoch 25 : 9517 / 10000
Epoch 26 : 9500 / 10000
Epoch 27 : 9507 / 10000
Epoch 28 : 9496 / 10000
Epoch 29 : 9505 / 10000


In [6]:
training_data , validation_data , test_data = load_data_wrapper()
net2 = Network([784, 100, 10])
net2.SGD(training_data , 30, 10, 3.0, test_data=test_data)

Epoch 0 : 6587 / 10000
Epoch 1 : 6763 / 10000
Epoch 2 : 6798 / 10000
Epoch 3 : 7697 / 10000
Epoch 4 : 7754 / 10000
Epoch 5 : 8569 / 10000
Epoch 6 : 8614 / 10000
Epoch 7 : 8655 / 10000
Epoch 8 : 8679 / 10000
Epoch 9 : 8670 / 10000
Epoch 10 : 8691 / 10000
Epoch 11 : 8688 / 10000
Epoch 12 : 8703 / 10000
Epoch 13 : 8699 / 10000
Epoch 14 : 8718 / 10000
Epoch 15 : 8715 / 10000
Epoch 16 : 8727 / 10000
Epoch 17 : 8731 / 10000
Epoch 18 : 8733 / 10000
Epoch 19 : 8745 / 10000
Epoch 20 : 8752 / 10000
Epoch 21 : 8746 / 10000
Epoch 22 : 8769 / 10000
Epoch 23 : 8751 / 10000
Epoch 24 : 8778 / 10000
Epoch 25 : 8770 / 10000
Epoch 26 : 8799 / 10000
Epoch 27 : 8835 / 10000
Epoch 28 : 8948 / 10000
Epoch 29 : 9623 / 10000


In [7]:
training_data , validation_data , test_data = load_data_wrapper()
net3 = Network([784, 30, 10])
net3.SGD(training_data , 30, 10, 0.001, test_data=test_data)

Epoch 0 : 614 / 10000
Epoch 1 : 607 / 10000
Epoch 2 : 561 / 10000
Epoch 3 : 649 / 10000
Epoch 4 : 947 / 10000
Epoch 5 : 1192 / 10000
Epoch 6 : 1304 / 10000
Epoch 7 : 1366 / 10000
Epoch 8 : 1410 / 10000
Epoch 9 : 1430 / 10000
Epoch 10 : 1483 / 10000
Epoch 11 : 1497 / 10000
Epoch 12 : 1530 / 10000
Epoch 13 : 1571 / 10000
Epoch 14 : 1622 / 10000
Epoch 15 : 1668 / 10000
Epoch 16 : 1700 / 10000
Epoch 17 : 1750 / 10000
Epoch 18 : 1792 / 10000
Epoch 19 : 1846 / 10000
Epoch 20 : 1898 / 10000
Epoch 21 : 1961 / 10000
Epoch 22 : 2002 / 10000
Epoch 23 : 2036 / 10000
Epoch 24 : 2082 / 10000
Epoch 25 : 2135 / 10000
Epoch 26 : 2172 / 10000
Epoch 27 : 2213 / 10000
Epoch 28 : 2256 / 10000
Epoch 29 : 2298 / 10000


In [8]:
training_data , validation_data , test_data = load_data_wrapper()
net4 = Network([784, 30, 10])
net4.SGD(training_data , 30, 10, 100.0, test_data=test_data)

Epoch 0 : 958 / 10000
Epoch 1 : 1080 / 10000
Epoch 2 : 1080 / 10000
Epoch 3 : 1080 / 10000
Epoch 4 : 1080 / 10000
Epoch 5 : 1080 / 10000
Epoch 6 : 1080 / 10000
Epoch 7 : 1080 / 10000
Epoch 8 : 1080 / 10000
Epoch 9 : 1080 / 10000
Epoch 10 : 1080 / 10000
Epoch 11 : 1080 / 10000
Epoch 12 : 1080 / 10000
Epoch 13 : 1080 / 10000
Epoch 14 : 1080 / 10000
Epoch 15 : 1080 / 10000
Epoch 16 : 1080 / 10000
Epoch 17 : 1080 / 10000
Epoch 18 : 1080 / 10000
Epoch 19 : 1080 / 10000
Epoch 20 : 1080 / 10000
Epoch 21 : 1080 / 10000
Epoch 22 : 1080 / 10000
Epoch 23 : 1080 / 10000
Epoch 24 : 1080 / 10000
Epoch 25 : 1080 / 10000
Epoch 26 : 1080 / 10000
Epoch 27 : 1080 / 10000
Epoch 28 : 1080 / 10000
Epoch 29 : 1080 / 10000


In [9]:
training_data , validation_data , test_data = load_data_wrapper()
net5 = Network([784, 30, 10])
net5.SGD(training_data , 30, 10, 10.0, test_data=test_data)

Epoch 0 : 9072 / 10000
Epoch 1 : 9157 / 10000
Epoch 2 : 9289 / 10000
Epoch 3 : 9296 / 10000
Epoch 4 : 9295 / 10000
Epoch 5 : 9335 / 10000
Epoch 6 : 9369 / 10000
Epoch 7 : 9356 / 10000
Epoch 8 : 9400 / 10000
Epoch 9 : 9420 / 10000
Epoch 10 : 9355 / 10000
Epoch 11 : 9454 / 10000
Epoch 12 : 9404 / 10000
Epoch 13 : 9467 / 10000
Epoch 14 : 9420 / 10000
Epoch 15 : 9446 / 10000
Epoch 16 : 9430 / 10000
Epoch 17 : 9446 / 10000
Epoch 18 : 9425 / 10000
Epoch 19 : 9400 / 10000
Epoch 20 : 9424 / 10000
Epoch 21 : 9473 / 10000
Epoch 22 : 9381 / 10000
Epoch 23 : 9442 / 10000
Epoch 24 : 9411 / 10000
Epoch 25 : 9469 / 10000
Epoch 26 : 9454 / 10000
Epoch 27 : 9505 / 10000
Epoch 28 : 9460 / 10000
Epoch 29 : 9478 / 10000


In [10]:
training_data , validation_data , test_data = load_data_wrapper()
net6 = Network([784, 100, 10])
net6.SGD(training_data , 30, 10, 0.1, test_data=test_data)

Epoch 0 : 4849 / 10000
Epoch 1 : 5776 / 10000
Epoch 2 : 6095 / 10000
Epoch 3 : 6227 / 10000
Epoch 4 : 6327 / 10000
Epoch 5 : 6366 / 10000
Epoch 6 : 6416 / 10000
Epoch 7 : 6443 / 10000
Epoch 8 : 6480 / 10000
Epoch 9 : 6538 / 10000
Epoch 10 : 6651 / 10000
Epoch 11 : 7294 / 10000
Epoch 12 : 7367 / 10000
Epoch 13 : 7409 / 10000
Epoch 14 : 7425 / 10000
Epoch 15 : 7434 / 10000
Epoch 16 : 7445 / 10000
Epoch 17 : 7462 / 10000
Epoch 18 : 7471 / 10000
Epoch 19 : 7477 / 10000
Epoch 20 : 7486 / 10000
Epoch 21 : 7493 / 10000
Epoch 22 : 7499 / 10000
Epoch 23 : 7503 / 10000
Epoch 24 : 7514 / 10000
Epoch 25 : 7517 / 10000
Epoch 26 : 7528 / 10000
Epoch 27 : 7546 / 10000
Epoch 28 : 7552 / 10000
Epoch 29 : 7567 / 10000
